In [30]:
import time
import numpy as np

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import torch
import torch.nn as nn
from torch.optim import Adam, SGD
import torch.optim.lr_scheduler as scheduler
from torch.utils.data import random_split,DataLoader

import torchvision
from torchvision.datasets import CIFAR10,CIFAR100
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor,Normalize

In [31]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
print(torch.cuda.current_device())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.device(0))

cuda
0
1
GeForce RTX 3090


In [32]:
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:0' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:',device)

True
학습을 진행하는 기기: cuda:0


# CIFAR-10

In [33]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [34]:
ds10=CIFAR10(root="./data",download=True,train=True,transform=transform)
ts10=CIFAR10(root="./data",download=True,train=False,transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [35]:
tr10,vs10,_=random_split(ds10,[10000,10000,30000],torch.manual_seed(1))

In [44]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=6,kernel_size=5,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        self.layer3 = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10),
            )
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        #out = out.view(out.size(0),-1) #Flatten feature mapes to one-dimension vector
        out = out.view(-1,16*5*5)
        out = self.layer3(out)
        return out
    
    def myForward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1,16*5*5)
        return x



In [45]:
myCnn=MyCNN().to(device)
training_epochs=20
batch_size=4
obj=nn.CrossEntropyLoss().to(device)
opt=SGD(myCnn.parameters(),lr=0.001, momentum=0.9)

In [46]:
tr10_loader=DataLoader(dataset=tr10,
                      batch_size=batch_size,
                      shuffle=True,
                      drop_last=True)

vs10_loader=DataLoader(dataset=vs10,
                      batch_size=batch_size,
                      shuffle=False)

In [47]:
myCnn=MyCNN().to(device)
training_epochs=8
batch_size=4
obj=nn.CrossEntropyLoss().to(device)
opt=SGD(myCnn.parameters(),lr=0.001, momentum=0.9)

In [48]:
start=time.time()

for epoch in range(training_epochs):
    training_loss=0.0
    validation_loss=0.0
    for X,y in tr10_loader:
        X=X.to(device)
        y=y.to(device)
        
        opt.zero_grad()
        output=myCnn(X) 
        cost=obj(output,y)
        cost.backward()
        opt.step()
        
print("Time :",time.time()-start)

Time : 62.079036712646484


In [49]:
tr10_loader_1=DataLoader(dataset=tr10,
                      batch_size=1,
                      shuffle=False)

In [51]:
image_tr_set=[]
label_tr_set=[]
for image, label in tr10_loader_1:
    #print(image.shape)
    image_tr_set.append(myCnn.myForward(image.to(device)).to('cpu').detach().numpy().reshape(400,))
    label_tr_set.append(int(label[0]))

In [52]:
RF=RandomForestClassifier()
RF.fit(image_tr_set,label_tr_set)

RandomForestClassifier()

In [53]:
ts10_loader_1=DataLoader(dataset=ts10,
                        batch_size=1,
                        shuffle=False)

In [54]:
image_ts_set=[]
label_ts_set=[]
for image, label in ts10_loader_1:
    #print(image.shape)
    image_ts_set.append(myCnn.myForward(image.to(device)).to('cpu').detach().numpy().reshape(400,))
    label_ts_set.append(int(label[0]))
    

In [55]:
pred=RF.predict(image_ts_set)
correct=0
for i in range(len(label_ts_set)):
    if pred[i]==label_ts_set[i]:
        correct+=1
print("Accuracy :",correct*100/10000)

Accuracy : 49.05


# CIFAR-100

In [56]:
ds100=CIFAR100(root="./data",download=True,train=True,transform=transform)
ts100=CIFAR100(root="./data",download=True,train=False,transform=transform)
tr100,vs100,_=random_split(ds100,[10000,10000,30000],torch.manual_seed(1))

Files already downloaded and verified
Files already downloaded and verified


In [57]:
tr100_loader=DataLoader(dataset=tr100,
                      batch_size=batch_size,
                      shuffle=True,
                      drop_last=True)

vs100_loader=DataLoader(dataset=vs100,
                      batch_size=batch_size,
                      shuffle=False)

In [58]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=6,kernel_size=5,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        self.layer3 = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,100),
            )
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        #out = out.view(out.size(0),-1) #Flatten feature mapes to one-dimension vector
        out = out.view(-1,16*5*5)
        out = self.layer3(out)
        return out
    
    def myForward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x.view(-1,16*5*5)


In [59]:
myCnn=MyCNN().to(device)
training_epochs=11
batch_size=4
obj=nn.CrossEntropyLoss().to(device)
opt=SGD(myCnn.parameters(),lr=0.001, momentum=0.9)

In [60]:
start=time.time()

for epoch in range(training_epochs):
    training_loss=0.0
    validation_loss=0.0
    for X,y in tr100_loader:
        X=X.to(device)
        y=y.to(device)
        
        opt.zero_grad()
        output=myCnn(X) 
        cost=obj(output,y)
        cost.backward()
        opt.step()
        
print("Time :",time.time()-start)

Time : 83.88899755477905


In [61]:
tr100_loader_1=DataLoader(dataset=tr100,
                      batch_size=1,
                      shuffle=False)

In [63]:
image100_tr_set=[]
label100_tr_set=[]
for image, label in tr100_loader_1:
    #print(image.shape)
    image100_tr_set.append(myCnn.myForward(image.to(device)).to('cpu').detach().numpy().reshape(400,))
    label100_tr_set.append(int(label[0]))

In [64]:
RF100=RandomForestClassifier()
RF100.fit(image100_tr_set,label100_tr_set)

RandomForestClassifier()

In [65]:
ts100_loader_1=DataLoader(dataset=ts100,
                        batch_size=1,
                        shuffle=False)

In [66]:
image100_ts_set=[]
label100_ts_set=[]
for image, label in ts100_loader_1:
    #print(image.shape)
    image100_ts_set.append(myCnn.myForward(image.to(device)).to('cpu').detach().numpy().reshape(400,))
    label100_ts_set.append(int(label[0]))
    

In [67]:
pred=RF100.predict(image100_ts_set)
correct=0
for i in range(len(label100_ts_set)):
    if pred[i]==label100_ts_set[i]:
        correct+=1
print("Accuracy :",correct*100/10000)

Accuracy : 19.13
